# **Project 3: Network and Sentiment Analysis of Yelp Restaurant Reviews?**    
**Group members: Jacob Silver, Justin Williams, Gregg Maloy**

Blah

**Part 1: Load Packages & Exploratory Analysis**   
Blah  

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import json

###file size is 4 gigs so i had to download to my machine and reference.  
##there are other files with different attributes but these are the two files which contain the info we need i think
user_json_path_business = 'C:/Users/maric/Documents/CUNYDataScience/620/final_project/yelp_academic_dataset_business.json'
user_json_path_review   = 'C:/Users/maric/Documents/CUNYDataScience/620/final_project/yelp_academic_dataset_review.json'

df_r = pd.read_json(user_json_path_review, lines=True)
df_b = pd.read_json(user_json_path_business, lines=True)

In [2]:
#this is the review file with user id, review id and business id 

print(df_r.head(10))

unique_review_ids = df_r['review_id'].nunique()
print("Number of unique review_ids:", unique_review_ids)

unique_business_id = df_r['business_id'].nunique()
print("Number of unique business_id:", unique_business_id)

unique_user_id = df_r['user_id'].nunique()
print("Number of unique user_id:", unique_user_id)

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   
5  JrIxlS1TzJ-iCu79ul40cQ  eUta8W_HdHMXPzLBBZhL1A  04UD14gamNjLY0IDYVhHJg   
6  6AxgBCNX_PNTOxmbRSwcKQ  r3zeYsv1XFBRA4dJpL78cw  gmjsEdUsKpj9Xxu6pdjH0g   
7  _ZeMknuYdlQcUqng_Im3yg  yfFzsLmaWF2d4Sr0UNbBgg  LHSTtnW3YHCeUkRDGyJOyw   
8  ZKvDG2sBvHVdF5oBNUOpAQ  wSTuiTk-sKNdcFyprzZAjg  B5XSoSG3SfvQGtKEGQ1tSQ   
9  pUycOfUwM8vqX7KjRRhUEA  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1    

In [3]:
##this is the business file which would allow us to filter by city or state if we want, as well if business is open or closed



print(df_b.head(10))

#unique_review_ids = df_b['review_id'].nunique()
#print("Number of unique review_ids:", unique_review_ids)

unique_business_id = df_b['business_id'].nunique()
print("Number of unique business_id:", unique_business_id)

#unique_user_id = df_b['user_id'].nunique()
#print("Number of unique user_id:", unique_user_id)

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
5  CF33F8-E6oudUQ46HnavjQ            Sonic Drive-In   
6  n_0UpQx1hsNbnPUSlodU8w           Famous Footwear   
7  qkRM_2X51Yqxk3btlwAQIg            Temple Beth-El   
8  k0hlBqXX-Bt0vf1op7Jr1w     Tsevi's Pub And Grill   
9  bBDDEgkFA1Otx9Lfe7BZUQ            Sonic Drive-In   

                                      address            city state  \
0                      1616 Chapala St, Ste 2   Santa Barbara    CA   
1             87 Grasso Plaza Shopping Center          Affton    MO   
2                        5255 E Broadway Blvd          Tucson    AZ   
3                                 935 Race St    Philadelphia    PA   
4                               101 Wal

In [4]:
#left joining reviewer file with business file in case we want to limit reviews to geographical area

merged_df = df_r.merge(df_b, how='left', on='business_id')

In [5]:
#descriptive stats of the new merged df

print(merged_df.head(10))

unique_review_ids = merged_df['review_id'].nunique()
print("Number of unique review_ids:", unique_review_ids)

unique_business_id = merged_df['business_id'].nunique()
print("Number of unique business_id:", unique_business_id)

unique_user_id = merged_df['user_id'].nunique()
print("Number of unique user_id:", unique_user_id)

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   
5  JrIxlS1TzJ-iCu79ul40cQ  eUta8W_HdHMXPzLBBZhL1A  04UD14gamNjLY0IDYVhHJg   
6  6AxgBCNX_PNTOxmbRSwcKQ  r3zeYsv1XFBRA4dJpL78cw  gmjsEdUsKpj9Xxu6pdjH0g   
7  _ZeMknuYdlQcUqng_Im3yg  yfFzsLmaWF2d4Sr0UNbBgg  LHSTtnW3YHCeUkRDGyJOyw   
8  ZKvDG2sBvHVdF5oBNUOpAQ  wSTuiTk-sKNdcFyprzZAjg  B5XSoSG3SfvQGtKEGQ1tSQ   
9  pUycOfUwM8vqX7KjRRhUEA  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg   

   stars_x  useful  funny  cool  \
0        3       0      0     0   
1        5       1      0     1   
2        3       0      0     0   
3        5  

In [6]:
#limit to those open restaurants only???? i dunno
# 1 = open, 0 = closed
merged_df = merged_df[merged_df['is_open']==1]

In [7]:
df_explode = merged_df.assign(categories=merged_df.categories.str.split(', ')).explode('categories')

# Count all categories
category_counts = df_explode['categories'].value_counts()

# Filter for restaurants only
restaurant_df = df_explode[df_explode['categories'].str.contains('Restaurants', case=True, na=False)]
restaurant_categories = restaurant_df['categories'].value_counts()

print('Total number of restaurant categories:', len(restaurant_categories))
print('Top 10 Restaurant Categories:')
#print(restaurant_categories[:10])
#print(restaurant_df[:10])

Total number of restaurant categories: 2
Top 10 Restaurant Categories:


In [32]:
##number of rows/reviews in df that where category coluymn is filtered to 'Restaurants'

num_rows = len(restaurant_df)
print("Number of rows:", num_rows)

Number of rows: 3777208


In [8]:
### here are the # of reviews by city. do we want to limit to one city to make more manageable?

grouped_restaurant = restaurant_df.groupby('city')
restaurant_count_by_city = grouped_restaurant.size().sort_values(ascending=False)
print(restaurant_count_by_city)

city
Philadelphia        511384
New Orleans         396258
Nashville           267392
Tampa               244494
Tucson              201342
                     ...  
Rural Hill               5
Oldmans Township         5
St Petersurg             5
Liverpool                5
Valencia West            5
Length: 846, dtype: int64


In [ ]:
##############i tried graphing the entire network and let it run for 30 min.
##############3 million rows is too much so we should probably limit to a city or something else

In [ ]:
######################BELOW IS A DIFFERENT ASSIGNMENT. I AM USING A THIS FILE AS A TEMPLATE 

In [ ]:
####STTTTTTTTTTTTTTTTTOOOOOOOOOPPPPPP

In [ ]:
####STOP

In [ ]:
#Create the bipartite graph
#graph = nx.Graph()


    
#plt.show()
#graph.add_nodes_from(restaurant_df['user_id']      ,bipartite=1, with_labels=False)
#graph.add_nodes_from(restaurant_df['business_id'],bipartite=0, with_labels=False)
##tesing data- for each node there is at least one edge
#print(nx.is_connected(graph))

##Returns True if graph G is bipartite, False if not.
#print(nx.is_bipartite(graph)) 
#node_colors = ['blue' if node_type == 1 else 'red' for node_type in nx.get_node_attributes(graph, 'bipartite').values()]

#for i in range(len(restaurant_df)):
#  graph.add_edges_from([(restaurant_df.iloc[i,1],restaurant_df.iloc[i,0])])

#plt.figure(figsize=(25,25))
#plt.text(0.5, 1.05, "reviewers (red) and restaurants (blue)", fontsize=24, ha='center', transform=plt.gca().transAxes)
#nx.draw_networkx(graph, with_labels=False, node_size=25,node_color=node_colors)
#plt.show()


   